### Load Context

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [24]:
data = catalog.load("data_proc")
data['len'] = data.groupby(['author', 'chapter']).feature.transform('count')
data = data[data.to_report]
data_stats = data[data.author.isin(['Dtr','DtrH','P'])].filter(['author', 'chapter', 'token_id', 'len'])
print("Number of docs per author:")
print(data_stats.groupby(['author']).chapter.nunique())

print("\nTotal no. of lemmas:")
print(data_stats.groupby('author').chapter.count())

print("\nMedian no. lemmas per chapter:")
print(data_stats.groupby('author').len.median())
#.agg(['count', 'mean', 'median']).loc[['Dtr', 'DtrH', 'P'],:]

print("\nTotal:")
print("Chapters: ", data_stats.chapter.nunique())
print("No. Lemmas: ", data_stats.chapter.count())
print("Median no. lemmas per chapter: ", data_stats.len.median())

2022-03-25 11:39:25,813 - kedro.io.data_catalog - INFO - Loading data from `data_proc` (CSVDataSet)...
Number of docs per author:
author
Dtr      9
DtrH    19
P       22
Name: chapter, dtype: int64

Total no. of lemmas:
author
Dtr      6324
DtrH    13410
P       14560
Name: chapter, dtype: int64

Median no. lemmas per chapter:
author
Dtr     568
DtrH    754
P       740
Name: len, dtype: int64

Total:
Chapters:  50
No. Lemmas:  34294
Median no. lemmas per chapter:  741.0


## Sim Full BS

In [8]:
probs_BS = catalog.load("probs_BS")
params = catalog.load('parameters')

2022-05-09 09:19:43,187 - kedro.io.data_catalog - INFO - Loading data from `probs_BS` (CSVDataSet)...
2022-05-09 09:19:43,210 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...


In [47]:
#from biblical_scripts.pipelines.reporting.nodes import summarize_probs

def _get_author_from_doc_id(st: str) -> str:
    return st.split('|')[0]

def max_value_cls_accuracy(dfr, value):
    idx_score = dfr.groupby('doc_tested')[value].idxmin()
    return (dfr.loc[idx_score, 'corpus'] == dfr.loc[idx_score, 'author']).mean()
    
    
def max_value_cls_miss(dfr, value):
    idx_score = dfr.groupby('doc_tested')[value].idxmin()
    dfr_res = dfr.loc[idx_score]
    dfr_miss = dfr_res[dfr_res['corpus'] != dfr_res['author']]
    return dfr_miss.filter(['doc_id', 'corpus', value])


def summarize_probs(dfm, params, log_scale=True):
    """
    Print summary from probabilities evaluated in comp_probs
    This function is mostly for debugging purposes.

    """

    if params['anova']:
        test_value_name = 'anova'
    else:
        test_value_name = 't_test'
    value = params['value']

    if log_scale:
        dfm.loc[:, 'sig_pval'] = np.exp(-dfm[test_value_name]) < params['sig_level']
    else:
        dfm.loc[:, 'sig_pval'] = dfm[test_value_name] < params['sig_level']
    print("========================RESULTS================================")

    dfm.loc[:, 'author'] = dfm.doc_tested.apply(_get_author_from_doc_id)
    dfr = dfm[dfm.author.isin(params['known_authors'])]
    
    acc_val = max_value_cls_accuracy(dfr, 'value')
    acc_test = max_value_cls_accuracy(dfr, test_value_name)
    miss_cls = max_value_cls_miss(dfr, test_value_name)
    
    print(f"Accuracy with {value}: ", acc_val)
    print(f"Accuracy with {test_value_name} of {value}: ", acc_test)
    print(f"Misclassified with {test_value_name}:")
    print(miss_cls)

    print(f"False Alarms with {test_value_name}:")
    print(dfr[(dfr.author == dfr.corpus) & dfr.sig_pval])
    
    df_res = dfr[dfr.author == dfr.corpus] \
        .groupby(['variable']) \
        .mean() \
        .filter(['sig_pval']) \
        .rename(columns={'sig_pval': f'FNR-{test_value_name}'})
    print(f"False alarm rate over {len(dfr)} chapters of known authorship:")
    print(df_res)
    
    return {'acc_value': acc_val,
            'acc_test': acc_test, 
            'miss_classified': miss_cls,
            'FNR': df_res.values[0]
           }


In [50]:
from biblical_scripts.pipelines.reporting.nodes import (
    add_stats_BS, add_stats_BS_full, comp_probs,
    summarize_probs_BS)


ImportError: cannot import name 'summarize_probs_BS' from 'biblical_scripts.pipelines.reporting.nodes' (/Users/kipnisal/DS/BiblicalScripts/bib-scripts/src/biblical_scripts/pipelines/reporting/nodes.py)

In [49]:
summarize_probs(probs_BS[probs_BS.itr_BS == 0], params['report'])

========================RESULTS================================
Accuracy with HC:  0.74
Accuracy with t_test of HC:  0.78
Misclassified with t_test:
           doc_id corpus    t_test
150   DtrH|1Kgs.8    Dtr  1.605507
346   DtrH|Deut.8    Dtr  0.251952
550  DtrH|Josh.12      P  0.073799
751  DtrH|Josh.23    Dtr  0.170757
391   DtrH|Judg.2    Dtr  0.162989
526   DtrH|Judg.6    Dtr  0.255370
726   Dtr|Deut.18   DtrH  0.325243
501    Dtr|Deut.6   DtrH  0.069830
787     P|Exod.16   DtrH  0.834331
246      P|Exod.6   DtrH  0.231962
802      P|Gen.17   DtrH  0.918125
False Alarms with t_test:
    author       doc_id     len variable     value   doc_tested  itr  smp_len  \
155   DtrH  DtrH|1Kgs.8  1764.0       HC  5.515573  DtrH|1Kgs.8   13      NaN   
156   DtrH  DtrH|1Kgs.8  1764.0       HC  5.515573  DtrH|1Kgs.8   13      NaN   
157   DtrH  DtrH|1Kgs.8  1764.0       HC  5.515573  DtrH|1Kgs.8   13      NaN   
158   DtrH  DtrH|1Kgs.8  1764.0       HC  5.515573  DtrH|1Kgs.8   13      NaN   


{'acc_value': 0.74,
 'acc_test': 0.78,
 'miss_classified':            doc_id corpus    t_test
 150   DtrH|1Kgs.8    Dtr  1.605507
 346   DtrH|Deut.8    Dtr  0.251952
 550  DtrH|Josh.12      P  0.073799
 751  DtrH|Josh.23    Dtr  0.170757
 391   DtrH|Judg.2    Dtr  0.162989
 526   DtrH|Judg.6    Dtr  0.255370
 726   Dtr|Deut.18   DtrH  0.325243
 501    Dtr|Deut.6   DtrH  0.069830
 787     P|Exod.16   DtrH  0.834331
 246      P|Exod.6   DtrH  0.231962
 802      P|Gen.17   DtrH  0.918125,
 'FNR': array([0.028])}

In [27]:
dfr.loc[idx_score]

,author,doc_id,len,variable,value,doc_tested,itr,smp_len,kind,checked_doc_len,...,rank,"('value', 'mean')","('value', 'std')","('value', 'median')","('value', 'mad')","('value', 'count')",rank_pval,t-score,t_test,sig_pval
1582,Ark1,Ark1|chapter0,1519.0,HC,4.397007,Ark1|chapter0,25,NaN,extended,1519.0,...,0.714286,4.098992,0.920233,3.940232,0.731968,19,0.285714,0.315648,0.972993,False
1506,Ark2,Ark2|chapter0,563.0,HC,3.542026,Ark2|chapter0,23,NaN,extended,563.0,...,0.380952,4.004462,0.754081,3.924705,0.609463,19,0.619048,-0.597717,0.326752,False
1159,Chr1,Chr1|chapter0,4689.0,HC,8.789799,Chr1|chapter0,2,NaN,extended,4689.0,...,0.909091,4.435566,1.362770,4.172090,0.870614,9,0.090909,3.031170,4.810940,True
1357,Chr2,Chr2|chapter0,3735.0,HC,6.011694,Chr2|chapter0,22,NaN,extended,3735.0,...,0.857143,4.228637,0.940925,3.978761,0.771089,19,0.142857,1.847022,3.203371,True
1081,Chr,Chr|chapter0,8357.0,HC,9.333658,Chr|chapter0,2,NaN,extended,8357.0,...,0.909091,4.701103,1.458046,4.377092,0.991615,9,0.090909,3.014190,4.785098,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,P,P|Lev.2,338.0,HC,3.619693,P|Lev.2,48,NaN,extended,338.0,...,0.217391,4.606331,0.853061,4.519145,0.646740,21,0.782609,-1.129994,0.146095,False
1678,P,P|Lev.3,411.0,HC,3.656637,P|Lev.3,44,NaN,extended,411.0,...,0.217391,4.589800,0.867207,4.516860,0.660507,21,0.782609,-1.051315,0.165849,False
970,P,P|Lev.4,859.0,HC,5.012319,P|Lev.4,31,NaN,extended,859.0,...,0.826087,4.506020,0.895241,4.677203,0.722005,21,0.173913,0.552542,1.226388,False
1378,P,P|Lev.8,855.0,HC,3.674019,P|Lev.8,38,NaN,extended,855.0,...,0.217391,4.604331,0.853598,4.517361,0.647310,21,0.782609,-1.064814,0.162311,False


In [29]:
dfr = probs_BS[probs_BS.itr_BS == 1]
value = 't_test'
idx_score = dfr.groupby('doc_tested')[value].idxmin()
dfr_res = dfr.loc[idx_score]
dfr_res[dfr_res['corpus'] != dfr_res['author']]['corpus']

1582    DtrH
1506    DtrH
1159     Dtr
1357    DtrH
1081     Dtr
1006     Dtr
1140     Dtr
1473     Dtr
1716    DtrH
996     DtrH
1185     Dtr
1432    DtrH
934      Dtr
1027    DtrH
1215     Dtr
1114     Dtr
1327    DtrH
1206    DtrH
1747    DtrH
Name: corpus, dtype: object

# Sim full  

In [2]:
sim_full_res = catalog.load("sim_full_res")
params = catalog.load('parameters')
print("len res = ",len(sim_full_res))

2022-02-26 02:05:48,183 - kedro.io.data_catalog - INFO - Loading data from `sim_full_res` (CSVDataSet)...
2022-02-26 02:05:48,198 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...
len res =  9948


In [3]:
from biblical_scripts.pipelines.reporting.nodes import (
    report_table_full_known, report_table_full_unknown,
    comp_probs, summarize_probs, _arrange_sim_full_results, _arrange_metadata
)

# Developing compute_probs:

In [14]:
sim_full_res = _arrange_sim_full_results(sim_full_res)
df = _arrange_metadata(sim_full_res, 'HC')

df0 = df[df.kind == 'generic']
df1 = df[df.kind != 'generic']

def ssquares(x):
    return np.sum((x - x.mean())**2)

dfss0 = df0[df0.author == df0.corpus].filter(['corpus', 'author', 'variable', 'value'])\
   .groupby(['variable', 'corpus'])\
    .agg(['mean', 'count', ssquares])
    
dfss1 = df1.filter(['author', 'variable', 'value', 'corpus', 'doc_id', 'doc_tested'])\
   .groupby(['doc_tested', 'variable', 'corpus'])\
    .agg(['mean', 'count', ssquares])

dfss = dfss1.join(dfss0, rsuffix='0', lsuffix='1')

from scipy.stats import f as fdist

def ftest(r):
    dfn = r[('value1', 'count')]
    dfd = r[('value0', 'count')]
    s = r[('value1', 'ssquares')] / r[('value0', 'ssquares')]
    return fdist.sf(s, dfn = dfn, dfd = dfd)
    
dfss['fpval'] = dfss.apply(ftest, axis=1)
dfr = dfss.reset_index().set_index('corpus')

In [101]:
#>>HERE!! fix ANOVA analysis, make sure that tests against own corpus make sense! (right now, they gives .5)

In [128]:
from scipy.stats import f as fdist

for r in dfss.iterrows():
    dfn = r[1][('value1', 'count')]
    dfd = r[1][('value0', 'count')]
    s = r[1][('value1', 'ssquares')] / r[1][('value0', 'ssquares')]
    print(r[0]," : ", fdist.sf(s, dfn = dfn, dfd = dfd))

('HC', 'Dtr', 'Dtr|Deut.12')  :  0.4999999999999999
('HC', 'Dtr', 'Dtr|Deut.13')  :  0.4999999999999999
('HC', 'Dtr', 'Dtr|Deut.15')  :  0.4999999999999999
('HC', 'Dtr', 'Dtr|Deut.16')  :  0.4999999999999999
('HC', 'Dtr', 'Dtr|Deut.18')  :  0.4999999999999999
('HC', 'Dtr', 'Dtr|Deut.19')  :  0.4999999999999999
('HC', 'Dtr', 'Dtr|Deut.26')  :  0.4999999999999999
('HC', 'Dtr', 'Dtr|Deut.28')  :  0.4999999999999999
('HC', 'Dtr', 'Dtr|Deut.6')  :  0.4999999999999999
('HC', 'DtrH', 'Dtr|Deut.12')  :  0.2994851749187902
('HC', 'DtrH', 'Dtr|Deut.13')  :  0.09198690504015013
('HC', 'DtrH', 'Dtr|Deut.15')  :  0.32845809349709926
('HC', 'DtrH', 'Dtr|Deut.16')  :  0.2108466478958217
('HC', 'DtrH', 'Dtr|Deut.18')  :  0.16027714156403888
('HC', 'DtrH', 'Dtr|Deut.19')  :  0.35600984324479173
('HC', 'DtrH', 'Dtr|Deut.26')  :  0.20119814469256328
('HC', 'DtrH', 'Dtr|Deut.28')  :  0.10224479327879214
('HC', 'DtrH', 'Dtr|Deut.6')  :  0.15324648306644864
('HC', 'P', 'Dtr|Deut.12')  :  0.20424979333465976

In [79]:
df.doc_id.unique()

array(['Ark1|chapter0', 'Dtr|Deut.12', 'Dtr|Deut.13', 'Dtr|Deut.15',
       'Dtr|Deut.16', 'Dtr|Deut.18', 'Dtr|Deut.19', 'Dtr|Deut.26',
       'Dtr|Deut.28', 'Dtr|Deut.6', 'DtrH|1Kgs.8', 'DtrH|2Kgs.17',
       'DtrH|2Kgs.22', 'DtrH|2Kgs.23', 'DtrH|2Kgs.24', 'DtrH|2Kgs.25',
       'DtrH|2Sam.7', 'DtrH|Deut.10', 'DtrH|Deut.11', 'DtrH|Deut.27',
       'DtrH|Deut.8', 'DtrH|Deut.9', 'DtrH|Josh.1', 'DtrH|Josh.12',
       'DtrH|Josh.23', 'DtrH|Josh.5', 'DtrH|Josh.6', 'DtrH|Judg.2',
       'DtrH|Judg.6', 'P|Exod.16', 'P|Exod.25', 'P|Exod.26', 'P|Exod.27',
       'P|Exod.28', 'P|Exod.29', 'P|Exod.30', 'P|Exod.31', 'P|Exod.35',
       'P|Exod.36', 'P|Exod.37', 'P|Exod.38', 'P|Exod.39', 'P|Exod.40',
       'P|Exod.6', 'P|Gen.17', 'P|Lev.1', 'P|Lev.2', 'P|Lev.3', 'P|Lev.4',
       'P|Lev.8', 'P|Lev.9', 'Ark2|chapter0', 'Chr1|chapter0',
       'Chr2|chapter0', 'Deut4|chapter0', 'Early_Jacob|chapter0',
       'Esth|chapter0', 'Gibea|chapter0', 'Late_Abraham|chapter0',
       'Lev26|chapter0', 'Prov|

In [64]:
dfss

value1                     value0        \
                                    mean count   ssquares      mean count   
variable corpus doc_tested                                                  
HC       Dtr    Ark1|chapter0   0.375323    10  14.454853  0.294129     9   
                Ark2|chapter0   0.399487    10   9.221976  0.294129     9   
                Chr1|chapter0   1.775739    10  24.629388  0.294129     9   
                Chr2|chapter0   1.084903    10  29.086314  0.294129     9   
                Deut4|chapter0  0.305294    10   3.629180  0.294129     9   
...                                  ...   ...        ...       ...   ...   
         P      DtrH|Josh.6     2.099531    23  15.768751  2.211519    22   
                DtrH|Judg.2     2.122001    23  15.824787  2.211519    22   
                DtrH|Judg.6     2.167873    23  17.533111  2.211519    22   
                Dtr|Deut.12     2.192777    23  17.735683  2.211519    22   
                Dtr|Deut.13     2.191613    23  13.586234  2.211519    22   

                                           
                                 ssquares  
variable corpus doc_tested                 
HC       Dtr    Ark1|chapter0    6.662417  
                Ark2|chapter0    6.662417  
                Chr1|chapter0    6.662417  
                Chr2|chapter0    6.662417  
                Deut4|chapter0   6.662417  
...                                   ...  
         P      DtrH|Josh.6     12.433359  
                DtrH|Judg.2     12.433359  
                DtrH|Judg.6     12.433359  
                Dtr|Deut.12     12.433359  
                Dtr|Deut.13     12.433359  

[80 rows x 6 columns]

#### Report

In [5]:
import plotnine
import logging
plotnine.options.figure_size = (10, 6)
from plotnine import *
logging.basicConfig(level=logging.INFO)

from biblical_scripts.pipelines.reporting.nodes import (
    report_table_full_known, report_table_full_unknown,
    comp_probs, summarize_probs, _arrange_sim_full_results
)

In [4]:
probs = comp_probs(sim_full_res, params['report'])

summarize_probs(probs, params['report'])

2022-01-02 13:31:24,274 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
========================RESULTS================================
Accuracy with ['HC']:  0.84
Accuracy with t score:  0.82
Miss classified with t score:
    author        doc_id     len variable     value    doc_tested  itr  \
54    DtrH   DtrH|1Kgs.8  1797.0       HC  4.150696   DtrH|1Kgs.8    0   
68    DtrH  DtrH|2Kgs.24   466.0       HC  3.719594  DtrH|2Kgs.24    0   
86    DtrH  DtrH|Josh.12   349.0       HC  3.288809  DtrH|Josh.12    0   
81    DtrH   DtrH|Josh.6   754.0       HC  3.930725   DtrH|Josh.6    0   
22     Dtr   Dtr|Deut.10   530.0       HC  1.907857   Dtr|Deut.10    0   
31     Dtr   Dtr|Deut.13   538.0       HC  2.310236   Dtr|Deut.13    0   
49     Dtr   Dtr|Deut.27   483.0       HC  2.220058   Dtr|Deut.27    0   
19     Dtr    Dtr|Deut.9   810.0       HC  2.746371    Dtr|Deut.9    0   
115      P     P|Exod.16   825.0       HC  4.174983     P|Exod.16    0   

     smp_len     kind  len

/usr/local/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
/usr/local/lib/python3.8/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.


,FNR-t,FNR-rank
variable,,
HC,0.04,0.02


In [76]:
var = 'HC'
df = sim_full_res[sim_full_res.variable.str.contains(':' + var)]
df['corpus'] = df.variable.str.extract(r"([a-zA-Z-]+):"+var)

df0 = df[~df.corpus.str.contains("-ext")]
df1 = df[df.corpus.str.contains("-ext")]
grp = df1.groupby(['author', 'doc_tested', 'corpus'])

value = 'value'
res = grp.agg({value: ['mean', 'std', 'count',
                       ]}, as_index=False).reset_index()



In [79]:
ref_data = catalog.load('reference_data')
print("number of P chapters to report = ", len(ref_data[(ref_data['author'] == 'P') & ref_data['to_report']]))

2021-12-29 14:02:59,677 - kedro.io.data_catalog - INFO - Loading data from `reference_data` (CSVDataSet)...
number of P chapters to report =  22


In [106]:
res_unknown = report_table_full_unknown(probs, params['report'])
res_known = report_table_full_known(probs, params['report'])

print("Known authors:")
print(res_known)

print("Unknown authors:")
print(res_unknown)

Known authors:
                HC                                                      \
doc_id DtrH|1Kgs.8 DtrH|2Kgs.17 DtrH|2Kgs.22 DtrH|2Kgs.23 DtrH|2Kgs.24   
corpus                                                                   
Dtr       4.150696     1.269382     3.510486     3.943187     3.115479   
DtrH      4.049639          1.0     1.708536     2.517542     2.599823   
P         7.012171     4.612525     4.537448     4.509708     4.100981   
succ          True         True         True         True         True   

                                                                       ...  \
doc_id DtrH|2Kgs.25 DtrH|2Sam.7 DtrH|Josh.1 DtrH|Josh.12 DtrH|Josh.23  ...   
corpus                                                                 ...   
Dtr        4.633836    2.889632         1.0      2.80626     1.170264  ...   
DtrH       2.513934    2.408426    1.352783     3.089592     1.981228  ...   
P           4.71206    5.735528    4.340428     2.876592     4.256065  ...  

In [107]:
res = report_table_full_known(probs, params['report'])
resHC = res['t-score'].T
np.mean(resHC.succ*1.)

0.86

In [110]:
dfm = probs.copy()
params['sig_level'] = .05
dfm.loc[:, 'sig_t'] = dfm['t_pval'] < params['sig_level']
dfm.loc[:, 'sig_rank'] = dfm['rank_pval'] < params['sig_level']

print("========================RESULTS================================")
dfr = dfm[dfm.author.isin(params['known_authors'])]
idx_var = dfr.groupby('doc_id')['value'].idxmin()
idx_tscore = dfr.groupby('doc_id')['t-score'].idxmin()
print(f"Accuracy with {dfr['variable'].unique()}: ",
      (dfr.loc[idx_var, 'corpus'] == dfr.loc[idx_var, 'author']).mean())
print(f"Accuracy with t score: ",
      (dfr.loc[idx_tscore, 'corpus'] == dfr.loc[idx_tscore, 'author']).mean())

print("Miss classified with t score:")
idx_miss = [idx for idx in idx_tscore if dfr.loc[idx, 'author']!=dfr.loc[idx,'corpus']]
print(dfr.loc[idx_miss, :])


========================RESULTS================================
Accuracy with ['HC']:  0.84
Accuracy with t score:  0.86
Miss classified with t score:
    author        doc_id    len variable     value    doc_tested  itr  \
75    DtrH   DtrH|Josh.1  494.0       HC  1.000000   DtrH|Josh.1    0   
86    DtrH  DtrH|Josh.12  349.0       HC  2.876592  DtrH|Josh.12    0   
87    DtrH  DtrH|Josh.23  518.0       HC  1.170264  DtrH|Josh.23    0   
90    DtrH   DtrH|Judg.2  574.0       HC  1.000000   DtrH|Judg.2    0   
49     Dtr   Dtr|Deut.27  483.0       HC  2.220058   Dtr|Deut.27    0   
115      P     P|Exod.16  825.0       HC  4.174983     P|Exod.16    0   
112      P      P|Exod.6  598.0       HC  2.757857      P|Exod.6    0   

     smp_len     kind  len_doc_tested  ... (value, mean)  (value, std)  \
75       494  generic             494  ...      1.918855      0.747817   
86       349  generic             349  ...      3.533714      0.953493   
87       518  generic             518  ...

In [16]:
from biblical_scripts.pipelines.reporting.nodes import _arrange_metadata, _pre_report_table_full

def report_table_full_unknown(sim_res_full, params_report, unknown_authors) :
    
    value = params_report['value']
    res = _arrange_metadata(sim_res_full, value)
    res = _pre_report_table_full(res)

    sig_level = params_report['sig_level']

    res_f=res[res.author.isin(unknown_authors)]
    lo_corpora = res.corpus.unique().tolist()

    res_tbl = res_f.pivot('corpus', 'doc_id', 'rnk_pval')
    cmin = res_tbl.idxmax().rename('max_pval_corpus')
    res_tbl = res_tbl.append(cmin)

    res_tbl.loc['author', :] = [r[1] for r in res_tbl.columns.str.split(' by ')]
    res_tbl.loc['significane', :] = res_tbl.loc[lo_corpora,:].max() > sig_level

    return res_tbl